In [11]:
from sklearn.datasets import load_files
from keras.utils import np_utils
from glob import glob
import numpy as np

In [12]:
def load_dataset(path):
    data = load_files(path)
    filenames = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 12)
    categories = np.array(data['target_names'])
    return filenames, targets, categories

#load dataset
train_filenames, y_targets, seedling_categories = load_dataset('seedling_data/train')
print('Total number of images: ', len(train_filenames)) #4750
#load list of categories
categories = seedling_categories
print('List of categories: ', seedling_categories)

#test_filenames, no_target, test_categories = load_dataset('seedling_data/test')
#print('Total number of images: ', len(test_filenames)) #4750

Total number of images:  4750
List of categories:  ['Black-grass' 'Charlock' 'Cleavers' 'Common Chickweed' 'Common wheat'
 'Fat Hen' 'Loose Silky-bent' 'Maize' 'Scentless Mayweed'
 'Shepherds Purse' 'Small-flowered Cranesbill' 'Sugar beet']


In [13]:
import matplotlib.pyplot as plt
'''
Function for displaying categorical data distribution to check if dataset is balanced
TODO: data augmentation if dataset is unbalanced
'''
category_count = np.sum(y_targets, axis = 0) # count of each class
def categorical_dist_graph(categories, count):
    fig, ax = plt.subplots()
    ax.bar(np.arange(len(categories)), count)
    ax.set_ylabel('No. of images per category')
    ax.set_xticks(np.arange(len(categories)))
    ax.set_xticklabels(categories, rotation = 'vertical')
    ax.set_title('Training data distribution')
    plt.show()

for category in categories:
    print("Category->{} has count".format(category))

categorical_dist_graph(categories, category_count)

TypeError: cannot perform reduce with flexible type

In [ ]:
import seaborn as sns

category_count = np.sum(y_targets, axis = 0) # count of each class
def categorical_dist_graph(categories, count):
    sns.set(color_codes=True)
    sns.distplot